## Important Notes
### Make sure to install unite with the editable flag.
### If you want 64-bit precision in JAX, you need to set the relevant environment variable: "JAX_ENABLE_X64" = "1"
### If you want to use exponential profiles, this is a hack for now, on line 327 of optimized.py replace integrateVoigt with integrateGaussianLaplace. Currently Cauchy/Exponential profiles cannot be used together.

In [ ]:
# some basic imports 

import astropy.units as u
import json
from astropy.io import fits
from astropy.table import Table, hstack,Column
from unite.spectra import NIRSpecSpectra
from unite.fitting import NIRSpecFit

/root/anaconda3/envs/linefit/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here is an example of running the code to fit two dispersers for a single spectrum. You create the table of the sources and ensure that they get loaded in correctly. To make sure different dispersers are processed together they must share the same root and srcid. 

In [ ]:
spec_names = ['rubies-egs53-nod-v3_prism-clear_4233_42046.spec.fits',
              'rubies-egs53-nod-v3_g395m-f290lp_4233_42046.spec.fits']
disps = ['prism'.upper(),'g395m'.upper()]
root = ['rubies-egs53-nod-v3']*len(disps)
grade = [3]*len(disps)
redshift = [5.2777]*len(disps)
srcid = [42046]*len(disps)
spec_table = Table(data=[srcid,spec_names,root,disps,grade,redshift],names=['srcid','file','root','grating','grade','z'])

# check to see what this table looks like
spec_table

srcid,file,root,grating,grade,z
int64,str51,str15,str5,int64,float64
42046,./rubies-egs53-v3_prism-clear_4233_42046.spec.fits,rubies-egs53-v3,PRISM,3,5.2777
42046,./rubies-egs53-v3_g395m-f290lp_4233_42046.spec.fits,rubies-egs53-v3,G395M,3,5.2777


In [ ]:
# we can test whether this table works by loading the NIRSpecSpectra class
# this will read in the spectra files and convert the data to f_lambda units automatically
# the two arguments needed are the table, and the directory where the spectra are stored
spectra_directory = 'spectra'
spectra = NIRSpecSpectra(rows=spec_table,spectra_directory=spectra_directory)


Now that we know how to load in the data, we can set up the fit itself. The emission lines (and their line types) that we want to fit need to be specified in a configuration file. 
Here you can also specify absorption or cauchy components. 

In [ ]:
# load in the config here
with open('example-config-narrow.json') as f:
    config = json.load(f)

Running the fit is now easy! We use MCMC sampling as implemented in numpyro NUTS. Let's use 500 samples for warm-up, and 1000 for the actual sampling.

The fitting produces some files that store the full chains and a summary of results, and also a figure showing the fit. You can choose the location where to store these files.

In [ ]:
# run the fit
NIRSpecFit(config, spec_table, spectra_directory=spectra_directory, output_directory='out', N=1000, num_warmup=500)


sample: 100%|█| 1500/1500 [00:05<00:00, 279.72it/s, 3 steps of size 4.56e-01. acc. prob=


* To change the *shape* of the priors we can edit the unite/priors.py file directly

* To change the *range* of the priors we can edit the unite/defaults.py file directly

* To change the width of the wavelength range used to fit, or the width used to find an initial estimate of the line fluxes, we can edit the unite/defaults.py file directly
